<a href="https://colab.research.google.com/github/learneverythingai/Shivam-Modi-Data-Science-Analytics-Course/blob/main/Deep%20Learning%20Course/Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The Author and Instructor of this Notebook is **Shivam Modi**.

## LinkedIn: https://www.linkedin.com/in/shivam-modi-datascientist/

# Tensorflow
Tensorlow is a deep learning library created by Google and is used to build, design and train deep learning models. The tensorflow operations in neural networks performed on multidimensional data arrays or tensors.

(reference: https://youtu.be/yjprpOoH5c8), use captions for better experience

## Installing tensorflow
import tensorflow as tf

In [ ]:
# Import `tensorflow`
import tensorflow as tf

# Initialize two constants
x1 = tf.constant([7,4,9,3])
x2 = tf.constant([1,8,2,4])

# Multiply
result = tf.multiply(x1, x2)

# Print the result
print(result)

tf.Tensor([ 7 32 18 12], shape=(4,), dtype=int32)


Tensorflow is used for:


1.   Loading and exploring the data
2.   Feature extraction
3.   Modeling Neural Network
4.   Evaluating the Neural Network



# Operations using Tensorflow
$Eager-execution$ is a powerful execution environment that evaluates operations immediately. It does not build graphs, and the operations return actual values instead of computational graphs to run later.

In [ ]:
import numpy as np
import tensorflow as tf

In [ ]:
print(tf.executing_eagerly()) #checking weather eager mode is enabled or not
print("Enabling eager mode!")

True
Enabling eager mode!


**if using tf version less than 2.0 then can be enabled using below comand line**
<p> tf.enable_eager_execution()

In [ ]:
# Define constant tensors
a = tf.constant(17)
print("a = %i" % a)
b = tf.constant(18)
print("b = %i" % b)

Define constant tensors
a = 17
b = 18


In [ ]:
# Run the operation without the need for tf.Session
c = a + b
print("a + b = %i" % c)
d = a * b
print("a * b = %i" % d)

Running operations, without tf.Session
a + b = 35
a * b = 306


In [ ]:
#Mixing operations with Tensors and Numpy Arrays
# Define constant tensors
a = tf.constant([[7., 4.],[0., 9.]], dtype=tf.float32)
print("Tensor:\n a = %s" % a)
b = np.array([[0., 6.],[4., 8.]], dtype=np.float32)
print("NumpyArray:\n b = %s" % b)

Tensor:
 a = tf.Tensor(
[[7. 4.]
 [0. 9.]], shape=(2, 2), dtype=float32)
NumpyArray:
 b = [[0. 6.]
 [4. 8.]]


In [ ]:
# Run the operation without the need for tf.Session
c = a + b
print("a + b = %s" % c)

d = tf.matmul(a, b)
print("a * b = %s" % d)

a + b = tf.Tensor(
[[ 7. 10.]
 [ 4. 17.]], shape=(2, 2), dtype=float32)
a * b = tf.Tensor(
[[16. 74.]
 [36. 72.]], shape=(2, 2), dtype=float32)


# Auto encoder with Tensorflow, keras
$Autoencoders$ are a type of unsupervised neural network helps to:
* Accept an input set of data (i.e., the input)
* Internally compress the input data and quantifies the input)
* Reconstruct the input data i.e., the output
<p> In general autoencoder have two components:
1. $Encoder$ : Accepts the input data and compresses it into the latent-space
2. $Decoder$ : The decoder is responsible for accepting the latent-space and then reconstructing the original input.

In [ ]:
from IPython import display

import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf
import tensorflow_probability as tfp
import time

In [ ]:
#loading the MNIST dataset
(train_images, _), (test_images, _) = tf.keras.datasets.mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


In [ ]:
#reshaping the images
def preprocess_images(images):
  images = images.reshape((images.shape[0], 28, 28, 1)) / 255.
  return np.where(images > .5, 1.0, 0.0).astype('float32')

train_images = preprocess_images(train_images)
test_images = preprocess_images(test_images)

In [ ]:
train_size = print(train_images.shape)
test_size = print(test_images.shape)
batch_size = 32

(60000, 28, 28, 1)
(10000, 28, 28, 1)


In [ ]:
train_size = 60000
test_size = 10000
batch_size = 32

In [ ]:
# Using tf.data to batch and shuffling the data
train_dataset = (tf.data.Dataset.from_tensor_slices(train_images).shuffle(train_size).batch(batch_size))
test_dataset = (tf.data.Dataset.from_tensor_slices(test_images).shuffle(test_size).batch(batch_size))

In [ ]:
class AE(tf.keras.Model):

  def __init__(self, latent_dim):
    super(AE, self).__init__()
    self.latent_dim = latent_dim # Decoder input is usually called latent embedding, and its dimension is latent_dim
    self.encoder = tf.keras.Sequential(
        [
            tf.keras.layers.InputLayer(input_shape=(28, 28, 1)),
            tf.keras.layers.Conv2D(
                filters=32, kernel_size=3, strides=(2, 2), activation='relu'),
            tf.keras.layers.Conv2D(
                filters=64, kernel_size=3, strides=(2, 2), activation='relu'),
            tf.keras.layers.Flatten(),
            # No activation
            tf.keras.layers.Dense(latent_dim + latent_dim),
        ]
    )

    self.decoder = tf.keras.Sequential(
        [
            tf.keras.layers.InputLayer(input_shape=(latent_dim,)),
            tf.keras.layers.Dense(units=7*7*32, activation=tf.nn.relu),
            tf.keras.layers.Reshape(target_shape=(7, 7, 32)),
            tf.keras.layers.Conv2DTranspose(
                filters=64, kernel_size=3, strides=2, padding='same',
                activation='relu'),
            tf.keras.layers.Conv2DTranspose(
                filters=32, kernel_size=3, strides=2, padding='same',
                activation='relu'),
            # No activation
            tf.keras.layers.Conv2DTranspose(
                filters=1, kernel_size=3, strides=1, padding='same'),
        ]
    )

  
  def sample(self, eps=None):
    if eps is None:
      eps = tf.random.normal(shape=(100, self.latent_dim))
    return self.decode(eps, apply_sigmoid=True)

  def encode(self, x):
    mean, logvar = tf.split(self.encoder(x), num_or_size_splits=2, axis=1)
    return mean, logvar

  def reparameterize(self, mean, logvar):
    eps = tf.random.normal(shape=mean.shape)
    return eps * tf.exp(logvar * .5) + mean

  def decode(self, z, apply_sigmoid=False):
    logits = self.decoder(z)
    if apply_sigmoid:
      probs = tf.sigmoid(logits)
      return probs
    return logits

In [ ]:
#defining loss function and optimizer
optimizer = tf.keras.optimizers.Adam(1e-4)


def log_normal_pdf(sample, mean, logvar, raxis=1):
  log2pi = tf.math.log(2. * np.pi)
  return tf.reduce_sum(-.5 * ((sample - mean) ** 2. * tf.exp(-logvar) + logvar + log2pi), axis=raxis)


def compute_loss(model, x):
  mean, logvar = model.encode(x)
  z = model.reparameterize(mean, logvar)
  x_logit = model.decode(z)
  cross_ent = tf.nn.sigmoid_cross_entropy_with_logits(logits=x_logit, labels=x)
  logpx_z = -tf.reduce_sum(cross_ent, axis=[1, 2, 3])
  logpz = log_normal_pdf(z, 0., 0.)
  logqz_x = log_normal_pdf(z, mean, logvar)
  return -tf.reduce_mean(logpx_z + logpz - logqz_x)


def train_step(model, x, optimizer):
  with tf.GradientTape() as tape:
    loss = compute_loss(model, x)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))